In [1]:
import pandas as pd
import numpy as np
import ast
import sys
print(sys.version_info)

sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)


In [2]:
dataname = "all-nodes"

In [4]:
data = pd.read_csv(dataname + ".csv")
data["children"] = data.children.apply(lambda s: list(ast.literal_eval(s)))

In [5]:
data

,id,name,productCount,subtreeProductCount,parent,numChildren,pathName,children,alsoCount,also
0,0,root,0,9354832,0,85,[],"[1, 2, 4, 10, 19, 22, 30, 34, 41, 44, 55, 60, ...",0,[]
1,1,Books,1940253,2370585,0,32,['Books'],"[45, 49, 52, 111, 118, 123, 134, 142, 155, 185...",0,[]
2,2,Movies & TV,1251,208321,0,2,['Movies & TV'],"[3, 27]",0,[]
3,3,Movies,142126,142126,2,0,"['Movies & TV', 'Movies']",[],428,"[(27, 591), (28, 552), (164, 526), (294, 341),..."
4,4,"Clothing, Shoes & Jewelry",0,1503384,0,225,"['Clothing, Shoes & Jewelry']","[5, 6, 231, 237, 379, 992, 1538, 1646, 1647, 1...",0,[]
...,...,...,...,...,...,...,...,...,...,...
29236,29236,Regents,1,1,21579,0,"['Books', 'Reference', 'Test Preparation', 'Co...",[],5,"[(16377, 1), (21413, 1), (16380, 1), (27674, 1..."
29237,29237,Regents,1,1,23683,0,"['Kindle Store', 'Kindle eBooks', 'Education &...",[],5,"[(16377, 1), (29236, 1), (21413, 1), (16380, 1..."
29238,29238,Law,1,1,25512,0,"['Kindle Store', 'Kindle eBooks', 'Foreign Lan...",[],10,"[(14243, 1), (20757, 1), (9277, 1), (17882, 1)..."
29239,29239,Solaris,1,1,6053,0,"['Books', 'Computers & Technology', 'Operating...",[],4,"[(14191, 1), (536, 1), (23218, 1), (16529, 1)]"


In [6]:
data["children"]

0        [1, 2, 4, 10, 19, 22, 30, 34, 41, 44, 55, 60, ...
1        [45, 49, 52, 111, 118, 123, 134, 142, 155, 185...
2                                                  [3, 27]
3                                                       []
4        [5, 6, 231, 237, 379, 992, 1538, 1646, 1647, 1...
                               ...                        
29236                                                   []
29237                                                   []
29238                                                   []
29239                                                   []
29240                                                   []
Name: children, Length: 29241, dtype: object

In [7]:
repeat_values = np.repeat(data["id"].values, data["children"].str.len())

In [8]:
df = data
lst_col = "children"
col = "id"

df[lst_col] = df[lst_col].apply(lambda x: x if len(x) > 0 else [np.nan])

r = pd.DataFrame({
      col:np.repeat(df[col].values, df[lst_col].str.len())
      for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(df[lst_col].values)})[df.columns]

r = r[['id', 'children', 'name'] + [c for c in df if c not in ['id', 'children', 'name']]]

In [9]:
r = r.astype({"children":'Int64'}, errors="ignore")

r = r.rename(columns={"id":"source", "children": "target", "name": "label"})

In [10]:
r.to_csv(dataname + '_edges.csv')

In [11]:
r

,source,target,label,productCount,subtreeProductCount,parent,numChildren,pathName,alsoCount,also
0,0,1,root,0,9354832,0,85,[],0,[]
1,0,2,root,0,9354832,0,85,[],0,[]
2,0,4,root,0,9354832,0,85,[],0,[]
3,0,10,root,0,9354832,0,85,[],0,[]
4,0,19,root,0,9354832,0,85,[],0,[]
...,...,...,...,...,...,...,...,...,...,...
53565,29236,NaN,Regents,1,1,21579,0,"['Books', 'Reference', 'Test Preparation', 'Co...",5,"[(16377, 1), (21413, 1), (16380, 1), (27674, 1..."
53566,29237,NaN,Regents,1,1,23683,0,"['Kindle Store', 'Kindle eBooks', 'Education &...",5,"[(16377, 1), (29236, 1), (21413, 1), (16380, 1..."
53567,29238,NaN,Law,1,1,25512,0,"['Kindle Store', 'Kindle eBooks', 'Foreign Lan...",10,"[(14243, 1), (20757, 1), (9277, 1), (17882, 1)..."
53568,29239,NaN,Solaris,1,1,6053,0,"['Books', 'Computers & Technology', 'Operating...",4,"[(14191, 1), (536, 1), (23218, 1), (16529, 1)]"
